In [ ]:
import findspark
findspark.init('/home/brunosouza/Documents/spark')

In [ ]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-sql-kafka-0-10_2.12:3.0.1 pyspark-shell'

In [ ]:
import sys
import time
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode
from pyspark.sql.functions import split

In [ ]:
spark = SparkSession \
    .builder \
    .appName("StructuredKafkaWordCount") \
    .getOrCreate()

In [ ]:
df = spark \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "localhost:9092,9093:9094") \
  .option("subscribe", "my-replicated-topic") \
  .load() \
  .selectExpr("CAST(value AS STRING)")

In [ ]:
words = df.select(
   explode(
       split(df.value, " ")
   ).alias("word")
)

In [ ]:
wordCounts = words.groupBy("word").count()

In [ ]:
query = wordCounts \
    .writeStream \
    .outputMode("complete") \
    .format("console") \
    .start()

In [ ]:
query.awaitTermination()